In [ ]:
import sys, os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.signal
import importlib
import h5py
sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
import readdata

In [ ]:
outfile='/Users/ajc/Core/PostProcessing/barrow_watershed/2021/January/'

In [ ]:
path_cades = "/Users/ajc/FUSE/simulations/production/barrow_watershed"
#path = "/Users/ajc/Core/simulations/projections/barrow_watershed"
path=path_cades
var_set = ['surface_flux-halfday.dat']

In [ ]:
S1 = [os.path.join(path, 'CESM_X/CESM_5050_R1%s_BX'%i) for i in range(1)] 
#S2 = [os.path.join(path, 'CESM_16/CESM_5050_R2%s_B'%i) for i in range(1)] 
S3 = [os.path.join(path, 'CESM_02/CESM_5050_R3%s_B'%i) for i in range(5)] 
#S4 = [os.path.join(path, 'CESM_16/CESM_5050_R4%s_B'%i) for i in range(1)] 
S5 = [os.path.join(path, 'CESM_02/CESM_5050_R5%s_BX'%i) for i in range(3)]

#Simulations = [S1, S2, S3, S4,S6]
Simulations = [S1, S3]


print (Simulations[1],)

In [ ]:
def read_file(file):
    d = np.loadtxt(file,usecols=1,skiprows=6)
    t1 = np.loadtxt(file,usecols=0,skiprows=6)
    t = np.array([x/86400./365. for x in t1])
    #print (len(d),len(t))
    l = np.minimum(len(d),len(t))
    return d[:l],t[:l]

def read_set(set,var):
    dat = []
    tim = []
    for s in set:
        file = os.path.join(path,s, var)
        #print (file)
        d = np.loadtxt(file,usecols=1,skiprows=6)
        t1 = np.loadtxt(file,usecols=0,skiprows=6)
        t = np.array([x/86400./365. for x in t1])
        
        L = len(d)%(365*2)
        if L >0:
            d = d[:-L]
            t = t[:len(d)]
            
        dat.append(d)
        tim.append(t)
    dat = np.concatenate(dat)
    tim = np.concatenate(tim)
    #print (len(dat)%365, len(tim)%365,L)
    
    return dat, tim

In [ ]:
path_sf ="/Users/ajc/Core/simulations/projections/barrow_watershed_02/CESM_15/CESM_5050_R60_B/visdump_surface_star_data.h5"
file_surf= h5py.File(path_sf,'r')
#print (file_surf.keys())
#print (file_surf['surface_star-cell_volume.cell.0'].keys())
basin_area = np.sum(list(file_surf['surface_star-cell_volume.cell.0']['100095']))
print (basin_area)

In [ ]:
#forcing = h5py.File("/Users/ajc/FUSE/simulations/production/barrow_watershed/data/CESM/CESM-RCP8_5-2006-2100_snowadjusted_supersmoothDM_2006.h5",'r')
forcing_watershed_FM = h5py.File("/Users/ajc/Core/PreProcessing/ats-intermediate-ngee/data/SNAP/CESM-RCP8.5-2006-2100_dm1985-2015-FM_2006.h5",'r')

In [ ]:
d1 = forcing_watershed_FM['precipitation rain [m s^-1]']
d2 = forcing_watershed_FM['precipitation snow [m SWE s^-1]']
rain_mm = np.reshape(d1,(-1,365)).mean(axis=1)*86400*365*1000
snow_mm = np.reshape(d2,(-1,365)).mean(axis=1)*86400*365*1000
precip_annual = snow_mm + rain_mm
r1 = np.array(d1)*86400*1000
r2 = np.array(d2)*86400*1000
precip_daily = r1 + r2

In [ ]:
key_var = ['surface_star-evaporative_flux','surface_star-depth','surface_star-carbon_decomposition_rate']

In [ ]:

def getSurfaceData(i,var):
    Data = []
    print ('getSurfaceData: ',var)
    for s in Simulations[i]:
        infile = os.path.join(s,'ats_vis_surface_star_data.h5')
        print (infile)
        dx=h5py.File(infile,'r')
        #print (dx.keys())
        d = np.array(readdata.GetSurfVarFromVis_All(infile,var)[var])
        d = d.mean(axis=1)
            
        L = len(d)%(365*2)
        if L >0:
            d = d[:-L]
            
        Data.append(d)
        #print (d.shape, len(d), L)
    d = dict()
    d[var] = np.concatenate(Data)
    
    return d

In [ ]:
#S1_Evap0 = getSurfaceData(0,key_var[0])

In [ ]:
#S1_Evap = getSurfaceData(1)

In [ ]:
def get_moving_average(d, m=5):
    d_avg = []
    len1 = len(d)
    for i in range(len1):
        if i < m:
            s1 = np.mean(d[:i+1])
        else:
            s1= np.mean(d[i-m:i])

        d_avg.append(s1)
    return d_avg

In [ ]:
colors = ['r','k','g','m','c','y']
marks = ['.','*', '+', 'o']
marks = ['-','-','-']
Labels = [r'S$_{1}$', r'S$_{2}$', r'S$_{3}$',r'S$_{4}$',r'S$_{6}$']
Labels = ['basecase','basecase-subsidence','static_sg','static_sg-subsidence','dynamic_sg', 'dynamic_sg (dst=0.2)']
Labels = ['Simplified','Basecase']

In [ ]:
#Slabels = ['S1', 'S2','S3','S4','S6','S6X']
Slabels = ['S1', 'S3','S5']
#Labels = ['S1_press', 'S3_hybrid', 'S3_press']
SimData = dict()
for i,sim in enumerate(Simulations):
    var = var_set[0]
    d1, t1 = read_set(sim,var)
    SimData[Slabels[i] + 'time'] = t1
    SimData[Slabels[i] + 'data'] = d2

In [ ]:
#Read discharge
Q = dict()
time = 2006 + np.linspace(0,94,94,dtype='int')
to_day_mm = 86400 * 1000
to_year_mm = 86400 * 365 * 1000
basin_area = 112635.48
to_mm = 1000.
for i,sim in enumerate(Simulations):
    var = var_set[0]
    
    d1, t1 = read_set(sim,var)
    d1 = [x/(55555.*basin_area)*to_day_mm for x in d1]
    
    d2 = np.reshape(d1,(-1,365*2)).mean(axis=1)*365
    t2 = np.reshape(t1,(-1,365*2)).mean(axis=1)
    #print (np.max(d2), len(d2), len(d1))
    d2 = get_moving_average(d2)
    Q[Slabels[i] + '_time'] = np.array(t2)
    Q[Slabels[i] + '_data'] = np.array(d2)

In [ ]:
def plot_Q():
    %matplotlib qt
    fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    #axs = axs.ravel()
    Colors = ['k','b','grey','g','r','m']
 
    axs.fill_between(1985 + Q['S6_time'], Q['S6_data']-0, Q['S6_data']*0+Q['S1_data'] , color='lightgray')
    axs.fill_between(1985 + Q['S6_time'], Q['S3_data'], Q['S6_data']+0 , color='gray')
    
    axs.plot(1985 + Q['S6_time'],Q['S6_data'],color=Colors[i],label=Labels[i],linestyle='dashed')
    axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
    axs.set_ylabel('Annual average discharge [mm]',fontsize=12, fontweight='normal')
    axs.legend(loc='upper right',fontsize=8)
    #axs.set_xlim(2005,2100)
    #axs.plot(time,precip_annual[:-1],'g')


    #axs.plot(t1[tt0*365:(tt0+1)*365]-21,precip_daily[tt0*365:(tt0+1)*365],'g')
    #plt.plot(rain_mm,'k')
    #plt.plot(snow_mm,'m')
    #plt.plot(precip_annual,'r')
    #print (outfile)
    #plt.savefig(outfile+'discharge-annualavg_bars.pdf', bbox_inches='tight', dpi=500)
    #print (outfile+'discharge-annualavg_bars.pdf')

In [ ]:
#plot_Q()

In [ ]:
%matplotlib qt
fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
    
plt.tight_layout()
#axs = axs.ravel()
Colors = ['k','r','grey','g','r','m']
time = 2006 + np.linspace(0,94,94,dtype='int')
to_day_mm = 86400 * 1000
to_year_mm = 86400 * 365 * 1000
basin_area = 112635.48
to_mm = 1000.
Discharge = dict()
for i,sim in enumerate(Simulations):
    var = var_set[0]
    
    d1, t1 = read_set(sim,var)
    d1 = [x/(55555.*basin_area)*to_day_mm for x in d1]
    
    d2 = np.reshape(d1,(-1,365*2)).mean(axis=1)*365
    t2 = np.reshape(t1,(-1,365*2)).mean(axis=1)
    Discharge[Labels[i]] = d2
    #print (np.max(d2), len(d2), len(d1))
    d2 = get_moving_average(d2)
    
    axs.plot(1985 + t2,d2,color=Colors[i],label=Labels[i],linestyle='dashed')
    axs.set_xlabel('Year',fontsize=12, fontweight='normal')
    axs.set_ylabel('Annual average discharge [mm]',fontsize=12, fontweight='normal')
    axs.legend(loc='upper right',fontsize=8)
    axs.set_xlim(2006,2100)
    #axs.plot(time,precip_annual[:-1],'g')
    #print (d2)
    
#axs.plot(t1[tt0*365:(tt0+1)*365]-21,precip_daily[tt0*365:(tt0+1)*365],'g')
#plt.plot(rain_mm,'k')
#plt.plot(snow_mm,'m')
#plt.plot(precip_annual,'r')
plt.savefig(outfile+'discharge-annualavg.pdf', bbox_inches='tight', dpi=500)

In [ ]:
[3.38848987 4.74302142 4.97081809 8.57229029 8.3188203  2.03423013
 2.81156703 1.64611619 8.24413518 5.1304231  6.67995395]
[0.83432683 0.68381174 1.09837145 3.04610905 4.02955726 0.00653959
 0.85990085 0.14517888 4.45851593 1.97298249 3.36534825 0.8067141 ]

In [ ]:
%matplotlib qt
fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
    
plt.tight_layout()
#axs = axs.ravel()
Colors = ['k','b','grey','g','r']
time = 2006 + np.linspace(0,94,94,dtype='int')
to_day_mm = 86400 * 1000
to_year_mm = 86400 * 365 * 1000
basin_area = 112635.48
to_mm = 1000.
for i,sim in enumerate(Simulations):
      
    #axs.plot(1985 + t2,d2,color=Colors[i],label=Labels[i])
    #axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
    #axs.set_ylabel('Annual average discharge [mm]',fontsize=12, fontweight='normal')
    var = key_var[2]
    d1 = getSurfaceData(i, var)[var]
    d3 = np.reshape(d1,(-1,365)).sum(axis=1)#*to_year_mm
    d3 = get_moving_average(d3)
    d3 = [x1/d3[0] for x1 in d3]
    #print (d3)
    axs.plot(d3,color=Colors[i],label=Labels[i], linestyle="dashed")
    axs.legend()
    
    
#axs.plot(t1[tt0*365:(tt0+1)*365]-21,precip_daily[tt0*365:(tt0+1)*365],'g')
#plt.plot(rain_mm,'k')
#plt.plot(snow_mm,'m')
#plt.plot(precip_annual,'r')
#plt.savefig(outfile+'carbon-decomp.pdf', bbox_inches='tight', dpi=500)

In [ ]:
fig, ax = plt.subplots()
X_perc = [4279, 4301, 6359, 6383,14866]
x=np.arange(len(X_perc))
ax.bar(x, X_perc,width=0.4)
ax.set_xticks(x)
ax.set_xticklabels(Labels)
ax.set_ylabel('Percentage increase [-]')
plt.savefig(outfile+'discharge-percentage_increace.pdf', bbox_inches='tight', dpi=800)

In [ ]:

def Evaporation():
    %matplotlib qt
    fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    #axs = axs.ravel()
    Colors = ['k','b','grey','g','r','m']
    time = 2006 + np.linspace(0,94,94,dtype='int')
    to_day_mm = 86400 * 1000
    to_year_mm = 86400 * 365 * 1000
    basin_area = 112635.48
    to_mm = 1000.
    Evap = dict()
    for i,sim in enumerate(Simulations):
        var = key_var[0]
        d1 = getSurfaceData(i, var)[var]
        l = len(d1)%365
        print (l)
        d3 = np.reshape(d1,(-1,365)).mean(axis=1)*to_year_mm
        Evap[Labels[i]] = d3
        #d3 = get_moving_average(d3)
        #print (d3)
        axs.plot(d3,color=Colors[i],label=Labels[i], linestyle="dashed")
        axs.legend(fontsize=8)
        #axs.plot(1985 + t2,d2,color=Colors[i],label=Labels[i])
        #axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
        #axs.set_ylabel('Annual average discharge [mm]',fontsize=12, fontweight='normal')

    #axs.plot(t1[tt0*365:(tt0+1)*365]-21,precip_daily[tt0*365:(tt0+1)*365],'g')
    #plt.plot(rain_mm,'k')
    #plt.plot(snow_mm,'m')
    #plt.plot(precip_annual,'r')
    #plt.savefig(outfile+'evaporation.pdf', bbox_inches='tight', dpi=500)
    return Evap

In [ ]:
Evap = Evaporation()
print (Evap)

In [ ]:
Including run-on in our simulations improve our understanding of total carbon budget. the evaporation will be under estimated
if we ignore run-on
long-term trend
inter-annual dynamics
seasonal vairations/dynamics
fire consumes carbon, which decrease DOC in stream

In [ ]:
def VolumetricWaterContent():
    %matplotlib qt
    fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    #axs = axs.ravel()
    Colors = ['k','b','grey','g','r']
    time = 2006 + np.linspace(0,94,94,dtype='int')
    to_day_mm = 86400 * 1000
    to_year_mm = 86400 * 365 * 1000
    basin_area = 112635.48
    to_mm = 1000.
    for i,sim in enumerate(Simulations):
        var_wc = 'surface_star-volumetric_wc'
        var_wc = 'surface_star-saturated_fraction'
        var_td = 'surface_star-thaw_depth'
        d1 = getSurfaceData(i, var_wc)[var_wc]
        d1_td = getSurfaceData(i, var_td)[var_td]
        #print (max(d1))
        d2 = np.reshape(d1,(-1,365))#.mean(axis=1)#*to_year_mm
        d2_td = np.reshape(d1_td,(-1,365))#.mean(axis=1)#*to_year_mm
        print (d2.shape)
        d3 = []
        for ix,iy in zip(d2,d2_td):
            x3 = np.where(ix >0)[0]
            x4 = np.where(iy >0)[0]
            sum1 = np.mean(ix[x3])/np.mean(x4)
            d3.append(sum1)
        
        d3 = get_moving_average(d3)
        d3 = [x/d3[0] for x in d3]
        #print (d3)
        axs.plot(d3,color=Colors[i],label=Labels[i], linestyle="dashed")
        axs.legend(fontsize=8)
        #axs.plot(1985 + t2,d2,color=Colors[i],label=Labels[i])
        #axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
        axs.set_ylabel('Volumetric water content [-]',fontsize=12, fontweight='normal')
        #break
    #axs.plot(t1[tt0*365:(tt0+1)*365]-21,precip_daily[tt0*365:(tt0+1)*365],'g')
    #plt.plot(rain_mm,'k')
    #plt.plot(snow_mm,'m')
    #plt.plot(precip_annual,'r')
    plt.savefig(outfile+'volumetric_wc.pdf', bbox_inches='tight', dpi=500)

In [ ]:
VolumetricWaterContent()

In [ ]:
def Average_ThawDepth():
    %matplotlib qt
    fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    #axs = axs.ravel()
    Colors = ['k','b','grey','g','r']
    time = 2006 + np.linspace(0,94,94,dtype='int')
    to_day_mm = 86400 * 1000
    to_year_mm = 86400 * 365 * 1000
    basin_area = 112635.48
    to_mm = 1000.
    for i,sim in enumerate(Simulations):
        #var_wc = 'surface_star-volumetric_wc'
        var_td = 'surface_star-thaw_depth'
        #d1 = getSurfaceData(i, var_wc)[var_wc]
        d1_td = getSurfaceData(i, var_td)[var_td]
        #print (max(d1))
        #d2 = np.reshape(d1,(-1,365))#.mean(axis=1)#*to_year_mm
        d2_td = np.reshape(d1_td,(-1,365))#.mean(axis=1)#*to_year_mm
        #print (d2.shape)
        d3 = []
        for ix in d2_td:
            x3 = np.where(ix >0)[0]
            #sum1 = np.mean(ix[x3])
            sum1 = np.max(ix[x3])
            d3.append(sum1)
        
        d3 = get_moving_average(d3)
        #d3 = [x/d3[0] for x in d3]
        #print (d3)
        axs.plot(d3,color=Colors[i],label=Labels[i], linestyle="dashed")
        axs.legend(fontsize=8)
        #axs.plot(1985 + t2,d2,color=Colors[i],label=Labels[i])
        #axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
        axs.set_ylabel('Averaged thaw depth []',fontsize=12, fontweight='normal')
        #break
    #axs.plot(t1[tt0*365:(tt0+1)*365]-21,precip_daily[tt0*365:(tt0+1)*365],'g')
    #plt.plot(rain_mm,'k')
    #plt.plot(snow_mm,'m')
    #plt.plot(precip_annual,'r')
    plt.savefig(outfile+'thaw_depth_max.pdf', bbox_inches='tight', dpi=500)

In [ ]:
Average_ThawDepth()

In [ ]:
def SaturatedFraction():
    %matplotlib qt
    fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    #axs = axs.ravel()
    Colors = ['k','b','grey','g','r']
    time = 2006 + np.linspace(0,94,94,dtype='int')
    to_day_mm = 86400 * 1000
    to_year_mm = 86400 * 365 * 1000
    basin_area = 112635.48
    to_mm = 1000.
    for i,sim in enumerate(Simulations):
        var_wc = 'surface_star-saturated_fraction'
        var_td = 'surface_star-thaw_depth'
        d1 = getSurfaceData(i, var_wc)[var_wc]
        #--d1_td = getSurfaceData(i, var_td)[var_td]
        #print (max(d1))
        d2 = np.reshape(d1,(-1,365))#.mean(axis=1)#*to_year_mm
        #--d2_td = np.reshape(d1_td,(-1,365))#.mean(axis=1)#*to_year_mm
        #print (d2.shape)
        d3 = []
        for ix in d2:
            x3 = np.where(ix >0)[0]
            sum1 = np.mean(ix[x3])
            d3.append(sum1)
        
        d3 = get_moving_average(d3)
        d3 = [x/d3[0] for x in d3]
        #print (d3)
        axs.plot(d3,color=Colors[i+2],label=Labels[i+2], linestyle="dashed")
        axs.legend(fontsize=8)
        #axs.plot(1985 + t2,d2,color=Colors[i],label=Labels[i])
        #axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
        axs.set_ylabel('Saturated fracton [-]',fontsize=12, fontweight='normal')
        #break
    #axs.plot(t1[tt0*365:(tt0+1)*365]-21,precip_daily[tt0*365:(tt0+1)*365],'g')
    #plt.plot(rain_mm,'k')
    #plt.plot(snow_mm,'m')
    #plt.plot(precip_annual,'r')
    plt.savefig(outfile+'saturated-fraction.pdf', bbox_inches='tight', dpi=500)

In [ ]:
SaturatedFraction()

In [ ]:
# 1) negligible differences in the discharge
# 2) significant differences in the evaporation, this implies base-case should stay relatively wetter,
#.   this will impact carbon decomposition rate
# 3) high NO3 and low N03, high DOC 

In [ ]:
def MassBalance():
    %matplotlib qt
    fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    #axs = axs.ravel()
    Colors = ['k','b','grey','g','r']
    time = 2006 + np.linspace(0,94,94,dtype='int')
    to_day_mm = 86400 * 1000
    to_year_mm = 86400 * 365 * 1000
    basin_area = 112635.48
    to_mm = 1000.
    Storage = dict()
    for i,sim in enumerate(Simulations):
        var_wc = 'surface_star-column_water_content'
        d1 = getSurfaceData(i, var_wc)[var_wc]
        #--d1_td = getSurfaceData(i, var_td)[var_td]
        #print (max(d1))
        d2 = np.reshape(d1,(-1,365))
    
        Storage[Labels[i]] = d2[0][0] - d[0][1]
        d3 = get_moving_average(d3)
        d3 = [x/d3[0] for x in d3]
        #print (d3)
        axs.plot(d3,color=Colors[i+2],label=Labels[i+2], linestyle="dashed")
        axs.legend(fontsize=8)
        #axs.plot(1985 + t2,d2,color=Colors[i],label=Labels[i])
        #axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
        axs.set_ylabel('Saturated fracton [-]',fontsize=12, fontweight='normal')
        #break
    #plt.savefig(outfile+'saturated-fraction.pdf', bbox_inches='tight', dpi=500)
    return Storage

In [ ]:
Storage = MassBalance()

In [ ]:
f = h5py.File('/Users/ajc/Core/simulations/projections/barrow_watershed/CESM_16/CESM_5050_R10_B/visdump_surface_star_data.h5','r')
f1 = h5py.File('/Users/ajc/Core/simulations/projections/barrow_watershed/CESM_15/CESM_5050_R10_B/visdump_surface_star_data.h5','r')

In [ ]:

keys = list(f.keys())
keys = list(f[keys[0]].keys())
print (keys[:3])

keys = sorted(keys, key=int)

In [ ]:
def read_set_cycle(set,var):
    dat = []
    tim = []
    for s in set:
        file = os.path.join(path,s, var)
        d = np.loadtxt(file,usecols=1,skiprows=6)
        t1 = np.loadtxt(file,usecols=0,skiprows=6)
        q = []
        t2 = []
        
        indx = np.where(t1%86400 == 43200.)[0]
        for i,x in enumerate(indx):
            if i ==0:
                q.append(0)
                t2.append(t1[0])
            else:
                sum1 =  np.mean(d[indx[i-1]:indx[i]])
                q.append(sum1)
                t2.append(t1[indx[i-1]])
                
        t = np.array([x/86400./365. for x in t2])
        
        d = q
        L = len(d)%365
        if L >0:
            d = d[:-L]
            t = t[:len(d)]
            
        dat.append(d)
        tim.append(t)
        print ('B:',len(t), len(d), L)
    dat = np.concatenate(dat)
    tim = np.concatenate(tim)
    
    return dat, tim

In [ ]:
%matplotlib qt
fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
    
plt.tight_layout()
#axs = axs.ravel()
Colors = ['k','b','grey','g','r','m']
time = 2006 + np.linspace(0,94,94,dtype='int')
to_day_mm = 86400 * 1000
to_year_mm = 86400 * 365 * 1000
basin_area = 112635.48
to_mm = 1000.
Discharge = dict()
for i,sim in enumerate(Simulations[:1]):
    d1, t1 = read_set_cycle(sim,'surface_flux-cycle.dat')
    d1 = [x/(55555.*basin_area)*to_day_mm for x in d1]
    
    #d2 = np.reshape(d1,(-1,365)).mean(axis=1)*365
    #t2 = np.reshape(t1,(-1,365)).mean(axis=1)
    
    d2 = np.reshape(d1,(-1,365))[0]#.mean(axis=1)*365
    t2 = np.reshape(t1,(-1,365))[0]#.mean(axis=1)
    
    d1A, t1A = read_set(sim,'surface_flux-day.dat')
    d1A = [x/(55555.*basin_area)*to_day_mm for x in d1A]
    
    #d2A = np.reshape(d1A,(-1,365)).mean(axis=1)*365
    #t2A = np.reshape(t1A,(-1,365)).mean(axis=1)
    
    d2A = np.reshape(d1A,(-1,365))[0]#.mean(axis=1)*365
    t2A = np.reshape(t1A,(-1,365))[0]#.mean(axis=1)
    
    axs.plot(1985 + t2,d2,color=Colors[i],label=Labels[i] + '-day',linestyle='solid')
    axs.plot(1985 + t2A,d2A,color='r',label=Labels[i] + '-cycle',linestyle='dotted')
    axs.legend()
    #axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
    #axs.set_ylabel('Annual average discharge [mm]',fontsize=12, fontweight='normal')

In [ ]:
2 meadian life time , 2.6hr give mean of 1hr median
alpha = 2.5 1/sec